In [2]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import CountVectorizer, StopWordsRemover, RegexTokenizer

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import string
from nltk.corpus import stopwords
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.types import StructField, StringType, StructType, IntegerType, ArrayType, LongType
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator 
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel, LogisticRegressionWithSGD
from pyspark.ml.classification import LogisticRegression, NaiveBayes 
from pyspark.ml.classification import RandomForestClassifier

from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col
from pyspark.sql import functions as F

from pyspark.sql.functions import udf
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

from pyspark.ml.feature import NGram
import sys
import os


SparkContext.setSystemProperty('spark.executor.memory', '6g')
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)


Import all files

In [4]:
trainD = pd.read_csv("train.csv")
# p_schema = [StructField('movie_id',LongType(),True),StructField('movie_name',StringType(),True),StructField('plot',StringType(),True),StructField('genre',ArrayType(),True)]
# Fschema = StructType(fields = p_schema)
# trainDS = spark.read.schema(p_schema).csv('train.csv')

trainDS = sqlContext.createDataFrame(trainD)
# trainDS.withColumn("genre", trainDS.genre.cast(ArrayType))

testD = pd.read_csv("test.csv")
testDS = sqlContext.createDataFrame(testD)



## Question 1 

Tokenize and use Counter Vectorizer here on train and test data

In [26]:
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="Wfiltered", stopWords = stopwords.words("english"))
ngram = NGram(n=2, inputCol="Wfiltered", outputCol="Nfiltered")
cv_tmp = CountVectorizer(inputCol="Nfiltered", outputCol="features", vocabSize=3000, binary=True, minDF=20.0)

pipelineCV = Pipeline(stages=[tokenizer,remover, ngram, cv_tmp])
featurizedData = pipelineCV.fit(trainDS)
df_vect = featurizedData.transform(trainDS)

#########################################################################################################

tokenizerTest = Tokenizer(inputCol="plot", outputCol="words")
removerTest = StopWordsRemover(inputCol="words", outputCol="Wfiltered", stopWords = stopwords.words("english"))
ngramTest = NGram(n=2, inputCol="Wfiltered", outputCol="Nfiltered")
cv_tmpTest = CountVectorizer(inputCol="Nfiltered", outputCol="features", vocabSize=3000, binary=True, minDF=10.0)

pipelineTest = Pipeline(stages=[tokenizerTest,removerTest, ngramTest, cv_tmpTest])
featurizedDataT = pipelineTest.fit(testDS)
df_vectT = featurizedDataT.transform(testDS)

df_vectT.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|           Wfiltered|           Nfiltered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|[film based, base...|(3000,[39,72,80,8...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|[group teenagers,...|(3000,[76,380],[1...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|[story zulu, zulu...|(3000,[460,514,70...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|[stooges play, pl...|(3000,[8,158,396,...|
|24165951|        Getting Even|A soldier-

Convert the labels in a list to get output

In [27]:
mapper = pd.read_csv('mapping.csv')
l1 = {}
l1 = [dict(zip(mapper.iloc[i].index.values, mapper.iloc[i].values)) for i in range(len(mapper))]

mapperS = sqlContext.createDataFrame(mapper)
mvv_array = [row.name for row in mapperS.collect()]

def Func(lines):
    
    lines = lines.strip('][').split(', ')
    p = [0] * 20
    for i in lines:
        for k,v in mapper.iterrows():
            if i == ('\''+v['name']+'\''):
                p[k] = 1
                continue
    k = str(p).strip('][\,')
    return str(k).replace(" ", "")

iplookup_udf = udf(Func)
df_vect = df_vect.withColumn("op", iplookup_udf("genre"))

In [28]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

newDS=df_vect.select("op","features")
newDS.printSchema()
# assembler = VectorAssembler(
#     inputCols=["features"],
#     outputCol="features1")

# transformed = assembler.transform(newDS)

root
 |-- op: string (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
df = newDS

split_col = pyspark.sql.functions.split(df['op'], ',')
df = df.withColumn('NAME1', split_col.getItem(0))
df = df.withColumn('NAME2', split_col.getItem(1))
df = df.withColumn('NAME3', split_col.getItem(2))
df = df.withColumn('NAME4', split_col.getItem(3))
df = df.withColumn('NAME5', split_col.getItem(4))
df = df.withColumn('NAME6', split_col.getItem(5))
df = df.withColumn('NAME7', split_col.getItem(6))
df = df.withColumn('NAME8', split_col.getItem(7))
df = df.withColumn('NAME9', split_col.getItem(8))
df = df.withColumn('NAME10', split_col.getItem(9))
df = df.withColumn('NAME11', split_col.getItem(10))
df = df.withColumn('NAME12', split_col.getItem(11))
df = df.withColumn('NAME13', split_col.getItem(12))
df = df.withColumn('NAME14', split_col.getItem(13))
df = df.withColumn('NAME15', split_col.getItem(14))
df = df.withColumn('NAME16', split_col.getItem(15))
df = df.withColumn('NAME17', split_col.getItem(16))
df = df.withColumn('NAME18', split_col.getItem(17))
df = df.withColumn('NAME19', split_col.getItem(18))
df = df.withColumn('NAME20', split_col.getItem(19))

df.show(5)

+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+
|                  op|            features|NAME1|NAME2|NAME3|NAME4|NAME5|NAME6|NAME7|NAME8|NAME9|NAME10|NAME11|NAME12|NAME13|NAME14|NAME15|NAME16|NAME17|NAME18|NAME19|NAME20|
+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+
|1,0,0,0,0,1,0,0,0...|  (3000,[608],[1.0])|    1|    0|    0|    0|    0|    1|    0|    0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|
|1,0,0,0,1,0,0,0,0...|(3000,[49,234,384...|    1|    0|    0|    0|    1|    0|    0|    0|    0|     0|     1|     0|     0|     0|     0|     0|     0|     1|     0|     0|
|1,0,0,0,1,0,0,0,0...|(3000,[1,2,48,72,...|    1|    0|    0|    0|    1|    0|    0|    0|    0|     0|     0|     0|     0|

In [30]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lit


listop = []
listl = ['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white',
         'Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction',
        'Mystery','Romantic comedy']


ops = pd.DataFrame()
ops['movie_id'] = [int(row.movie_id) for row in df_vectT.collect()]

for i in range(len(listl)):
    dramadf = df.select('features','NAME'+str(i+1))
    dramadf = dramadf.withColumnRenamed('NAME'+str(i+1),'label')
    dramadf = dramadf.withColumn("label", dramadf["label"].cast(IntegerType()))
    
    ################ set pipeline ########################
    lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=8)
    modelN = lr.fit(dramadf)
    modelN.save(os.path.join(sys.argv[1], 'model_'+str(i+1)))
    predictionsN = modelN.transform(df_vectT)
    
    ################ store ouputs offline ##################
    mvv_array = [int(row.prediction) for row in predictionsN.collect()]
    ops['op'+str(i+1)] = mvv_array

cols = ['op1','op2','op3','op4','op5','op6','op7','op8','op9','op10',
        'op11','op12','op13','op14','op15','op16','op17','op18','op19','op20']


ops.head()
FinalOutput = pd.DataFrame() 
FinalOutput['movie_id'] = ops['movie_id']
FinalOutput['predictions'] = ops[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
FinalOutput.to_csv('OutputDemo.csv')

### Check F1 score for the models

In [150]:
F1DF = df.select('features','NAME2')
F1DF = F1DF.withColumnRenamed('NAME2','label')
F1DF = F1DF.withColumn("label", F1DF["label"].cast(IntegerType()))
training, testing = F1DF.randomSplit([0.7,0.3], seed=100)

In [151]:

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(training)
predictions = lrModel.transform(testing)

# (c.select(col("label").alias("label"), col("features1"))
#   .rdd
#   .map(lambda row: LabeledPoint(row.label, row.features)))
# temp = newDS.map(lambda line:LabeledPoint(line[7],[line[6]]))
# temp.take(5)

# from pyspark.ml.evaluation import ClusteringEvaluator
# predictions = model.transform(trainingData)
# evaluator = ClusteringEvaluator()
# centers = model.clusterCenters()
# silhouette = evaluator.evaluate(predictions)
# print("Silhouette with squared euclidean distance = " + str(silhouette))

# model = RandomForestClassifier(featuresCol = 'features', labelCol='label')
# rf_model = model.fit(newDS)

In [152]:
evaluator_cv_lr = MulticlassClassificationEvaluator().setPredictionCol("prediction").evaluate(predictions)
print('                       accuracy:{}:'.format(evaluator_cv_lr))

                       accuracy:0.6927875547085338:


## Question 2 

Applying the TF-IDF for vectorizing the inputs better

In [16]:
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="Wfiltered")
# ngram = NGram(n=2, inputCol="Wfiltered", outputCol="Nfiltered")

hashingTF = HashingTF(inputCol="Wfiltered", outputCol="hashed_features", numFeatures=10000)
idf_stage = IDF(inputCol="hashed_features", outputCol="features", minDocFreq=4)

pipelineCV = Pipeline(stages=[tokenizer,remover,hashingTF, idf_stage])
featurizedtrainTF = pipelineCV.fit(trainDS)
df_trianTDF = featurizedtrainTF.transform(trainDS)


featurizedtestTF = pipelineCV.fit(trainDS)
df_testTDF = featurizedtestTF.transform(testDS)



In [17]:
df_vectTDF = df_trianTDF.withColumn("op", iplookup_udf("genre"))


In [20]:
# from pyspark.ml.classification import LinearSVC
# F1DFTF = dfTF.select('features','NAME2')
# F1DFTF = F1DFTF.withColumnRenamed('NAME2','label')
# F1DFTF = F1DFTF.withColumn("label", F1DFTF["label"].cast(IntegerType()))
# trainingTF, testingTF = F1DFTF.randomSplit([0.7,0.3], seed=100)

# lrTF = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
# lrModelTF = lrTF.fit(trainingTF)
# predictionsTF = lrModelTF.transform(testingTF)

# evaluator_cv_lrTF = MulticlassClassificationEvaluator().setPredictionCol("prediction").evaluate(predictionsTF)
# print('                       accuracy:{}:'.format(evaluator_cv_lrTF))

In [19]:
df_vectTDF = df_vectTDF.withColumn("op", iplookup_udf("genre"))
NewdfTF = df_vectTDF.select('features','op')
split_col = pyspark.sql.functions.split(NewdfTF['op'], ',')
for i in range(20):
    NewdfTF = NewdfTF.withColumn('NAME'+str(i+1), split_col.getItem(i))

NewdfTF.show()


+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+
|            features|                  op|NAME1|NAME2|NAME3|NAME4|NAME5|NAME6|NAME7|NAME8|NAME9|NAME10|NAME11|NAME12|NAME13|NAME14|NAME15|NAME16|NAME17|NAME18|NAME19|NAME20|
+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+
|(10000,[135,265,7...|1,0,0,0,0,1,0,0,0...|    1|    0|    0|    0|    0|    1|    0|    0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|
|(10000,[26,47,66,...|1,0,0,0,1,0,0,0,0...|    1|    0|    0|    0|    1|    0|    0|    0|    0|     0|     1|     0|     0|     0|     0|     0|     0|     1|     0|     0|
|(10000,[52,74,104...|1,0,0,0,1,0,0,0,0...|    1|    0|    0|    0|    1|    0|    0|    0|    0|     0|     0|     0|     0|

In [21]:
########################################## TF IDF #################################################
path = 'model_rfcTF'
os.mkdir(path)

listop = []
listl = ['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white',
         'Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction',
        'Mystery','Romantic comedy']
opsTF = pd.DataFrame()
opsTF['movie_id'] = [int(row.movie_id) for row in df_testTDF.collect()]

for i in range(len(listl)):
    dramadf = NewdfTF.select('features','NAME'+str(i+1))
    dramadf = dramadf.withColumnRenamed('NAME'+str(i+1),'label')
    dramadf = dramadf.withColumn("label", dramadf["label"].cast(IntegerType()))
    
    ################ set pipeline ########################
    lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=8)
    modelN = lr.fit(dramadf)
    modelN.save(os.path.join(sys.argv[1], 'model_'+str(i+1)))
    predictionsN = modelN.transform(df_testTDF)
    
    ################ store ouputs offline ##################
    mvv_array = [int(row.prediction) for row in predictionsN.collect()]
    opsTF['op'+str(i+1)] = mvv_array

colsTF = ['op1','op2','op3','op4','op5','op6','op7','op8','op9','op10',
        'op11','op12','op13','op14','op15','op16','op17','op18','op19','op20']

opsTF.head()
FinalOutputTF = pd.DataFrame() 
FinalOutputTF['movie_id'] = opsTF['movie_id']
FinalOutputTF['predictions'] = opsTF[colsTF].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
FinalOutputTF.to_csv('OutputTFNew.csv')

## Question 3

## Applying Word to Vector

In [6]:
from pyspark.ml.feature import Word2Vec
from nltk.stem.snowball import SnowballStemmer

tokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
tokenDF = tokenizer.transform(trainDS)

remover = StopWordsRemover(inputCol="words", outputCol="Wfiltered")
removeDF = remover.transform(tokenDF)

stem = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tks: [stem.stem(row) for row in tks], ArrayType(StringType()))
stemmed = removeDF.withColumn("Lfilter", stemmer_udf("Wfiltered"))

word2Vec = Word2Vec(vectorSize=250, seed=43, inputCol="Lfilter", outputCol="features", minCount=5)

pipelineW2V = Pipeline(stages=[word2Vec])
featurizedtrainW2V = pipelineW2V.fit(stemmed)
df_trianW2V = featurizedtrainW2V.transform(stemmed)


tokenDFTest = tokenizer.transform(testDS)
removeDFTest = remover.transform(tokenDFTest)
stemmed = removeDFTest.withColumn("Lfilter", stemmer_udf("Wfiltered"))

featurizedtestW2V = pipelineW2V.fit(stemmed)
df_testW2V = featurizedtestW2V.transform(stemmed)

df_trianW2V.show(5)

+--------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|        movie_name|                plot|               genre|               words|           Wfiltered|             Lfilter|            features|
+--------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|        Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[shlykov, hard, w...|[0.03008223511963...|
|31186339|  The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[nation, panem, c...|[0.04306361994775...|
|20663735|        Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[pooval, induchoo...|[-0.0210946972665...|
| 2231378|The Lemon Drop Kid|The Lemon Drop Ki...|          ['Co

In [9]:
df_vectW2V = df_trianW2V.withColumn("op", iplookup_udf("genre"))

In [10]:
df_vectW2V.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|           Wfiltered|             Lfilter|            features|                  op|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[shlykov, hard, w...|[0.03008223511963...|1,0,0,0,0,1,0,0,0...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[nation, panem, c...|[0.04306361994775...|1,0,0,0,1,0,0,0,0...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, 

In [11]:
df_vectW2V = df_vectW2V.withColumn("op", iplookup_udf("genre"))
NewdfW2V = df_vectW2V.select('features','op')
split_col = pyspark.sql.functions.split(NewdfW2V['op'], ',')
for i in range(20):
    NewdfW2V = NewdfW2V.withColumn('NAME'+str(i+1), split_col.getItem(i))

NewdfW2V.show()

+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+
|            features|                  op|NAME1|NAME2|NAME3|NAME4|NAME5|NAME6|NAME7|NAME8|NAME9|NAME10|NAME11|NAME12|NAME13|NAME14|NAME15|NAME16|NAME17|NAME18|NAME19|NAME20|
+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+
|[0.03008223511963...|1,0,0,0,0,1,0,0,0...|    1|    0|    0|    0|    0|    1|    0|    0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|
|[0.04306361994775...|1,0,0,0,1,0,0,0,0...|    1|    0|    0|    0|    1|    0|    0|    0|    0|     0|     1|     0|     0|     0|     0|     0|     0|     1|     0|     0|
|[-0.0210946972665...|1,0,0,0,1,0,0,0,0...|    1|    0|    0|    0|    1|    0|    0|    0|    0|     0|     0|     0|     0|

In [31]:
path = 'model_rfcW2V'
os.mkdir(path)

listop = []
listl = ['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white',
         'Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction',
        'Mystery','Romantic comedy']

opsW2V = pd.DataFrame()
opsW2V['movie_id'] = [int(row.movie_id) for row in df_testW2V.collect()]

for i in range(len(listl)):
    dramadf = NewdfW2V.select('features','NAME'+str(i+1))
    dramadf = dramadf.withColumnRenamed('NAME'+str(i+1),'label')
    dramadf = dramadf.withColumn("label", dramadf["label"].cast(IntegerType()))
    
    ################ set pipeline ########################
    lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=30,family = "binomial")
    
    modelN = lr.fit(dramadf)
    modelN.save(os.path.join(sys.argv[1], 'model_'+str(i+1)))
    predictionsW2V = modelN.transform(df_testW2V)
    ################ store ouputs offline ##################
    mvv_array = [int(row.prediction) for row in predictionsW2V.collect()]
    opsW2V['op'+str(i+1)] = mvv_array

colsW2V = ['op1','op2','op3','op4','op5','op6','op7','op8','op9','op10',
        'op11','op12','op13','op14','op15','op16','op17','op18','op19','op20']

opsW2V.head()
FinalOutputW2V = pd.DataFrame() 
FinalOutputW2V['movie_id'] = opsW2V['movie_id']
FinalOutputW2V['predictions'] = opsW2V[colsW2V].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
FinalOutputW2V.to_csv('OutputW2VNew.csv')

### Demo run to test accuracy on one genre

In [32]:
F1DFW2V = NewdfW2V.select('features','NAME2')
F1DFW2V = F1DFW2V.withColumnRenamed('NAME2','label')
F1DFW2V = F1DFW2V.withColumn("label", F1DFW2V["label"].cast(IntegerType()))
trainingW2V, testingW2V = F1DFW2V.randomSplit([0.7,0.3], seed=100)

lrW2V = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=25, family = "binomial")
lrModelW2V = lrW2V.fit(trainingW2V)
predictionsW2V = lrModelW2V.transform(testingW2V)


In [33]:

evaluator_cv_lrTF = MulticlassClassificationEvaluator().setPredictionCol("prediction").evaluate(predictionsW2V)
print('                       accuracy:{}:'.format(evaluator_cv_lrTF))

                       accuracy:0.7339256326311449:


## To load the saved models and to get predictions

1. Change the path to the 'modelsQ1' or 'modelsQ2' or 'modelsQ3' folder as required
2. Update the Variable names of DataFrame variable 'DATAIN' to any one of the following as required.
                Question 1 : df_vectT
                Question 2 : df_testTDF
                Question 3 : df_testW2V
    

In [25]:
from pyspark.ml.classification import LogisticRegressionModel


listl = ['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white',
         'Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction',
        'Mystery','Romantic comedy']

### Update the following variable 
DATAIN = df_testTDF

outputPd = pd.DataFrame()
outputPd['movie_id'] = [int(row.movie_id) for row in DATAIN.collect()]

for i in range(len(listl)):
    k = i+1
    
    #### Update the path as required
    LogisticRegressionModel.load('/home/cse587/Project3/modelsQ2/'+'model_'+str(k))
    predictionsN = modelN.transform(DATAIN)
    mvv_array = [int(row.prediction) for row in predictionsN.collect()]
    outputPd['op'+str(i+1)] = mvv_array

cols = ['op1','op2','op3','op4','op5','op6','op7','op8','op9','op10',
        'op11','op12','op13','op14','op15','op16','op17','op18','op19','op20']


FinalOutputTF = pd.DataFrame() 
FinalOutputTF['movie_id'] = outputPd['movie_id']
FinalOutputTF['predictions'] = outputPd[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
FinalOutputTF.to_csv('QuestionOutput.csv')

    